In [4]:
%pip install spotlight

  Using cached spotlight-3.3.0-py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install rdflib

     -------------------------------------- 531.9/531.9 kB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 41.7/41.7 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.1.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import spotlight
from rdflib import Graph, Literal, URIRef, Namespace
from spotlight import exceptions as SpotlightException

def annotate(ttl_file_path, spotlight_api_url, output_ttl_file):
    # Load TTL data into an RDF graph
    graph = Graph()
    with open(ttl_file_path, 'r', encoding='utf-8') as ttl_file:
        graph.parse(ttl_file, format='turtle')

    # Create a namespace for the new predicate
    ns = Namespace("http://example.org/")

    # Open the output file for writing
    with open(output_ttl_file, 'w') as output_file:
        # Iterate over triples in the original graph
        for subject, predicate, obj in graph:
            # Write the original triple to the output file
            output_file.write(f"{subject} {predicate} {obj} .\n")

            # If the predicate is 'http://schema.org/category', annotate and append the new attribute
            if predicate == URIRef('http://schema.org/category') and isinstance(obj, Literal):
                category_value = obj.value

                try:
                    # Annotate the category value using DBpedia Spotlight
                    annotations = spotlight.annotate('https://api.dbpedia-spotlight.org/en/annotate?types={DBpedia:Food}', category_value, confidence=0.6)

                    # Extract the URI of the 'category' property from the annotations
                    category_uri = None
                    category_uri = annotations[0]['URI']

                    # Check if a valid URI was found before adding the new attribute
                    if category_uri is not None:
                        # Append a new attribute with the subject, new predicate, and URI value
                        new_attribute = (subject, ns.deducedCategory, URIRef(category_uri))

                        # Write the new attribute to the output file
                        output_file.write(f"{new_attribute} .\n")

                except Exception as e:
                    # Handle the exception (no resources found in spotlight response)
                    print(f"Warning: No resources found for '{category_value}'. {e}")

In [43]:
import pandas as pd
import spotlight
from spotlight import exceptions as SpotlightException

def annotate_category(row):
    category_value = row['category']

    try:
        # Annotate the category value using DBpedia Spotlight
        annotations = spotlight.annotate('https://api.dbpedia-spotlight.org/en/annotate?types={DBpedia:Food}', category_value, confidence=0.6)

        # Extract the URI of the 'category' property from the annotations
        category_uri = annotations[0]['URI'] if annotations else None

        return category_uri

    except Exception as e:
        # Handle the exception (no resources found in spotlight response)
        print(f"Warning: No resources found for '{category_value}'. {e}")
        return None

In [44]:
# Input and output file paths
csv_file_path = '../csvw/walmart_products.csv'
output_csv_file = '../csvw/walmart_products_annotated.csv'
spotlight_api_url = 'https://api.dbpedia-spotlight.org/en/annotate?types=DBpedia:Food'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Apply the annotation function to each row and create the 'deducedType' column
df['deducedType'] = df.apply(annotate_category, axis=1)

# Save the updated DataFrame to a new CSV file
df.to_csv(output_csv_file, index=False)

KeyboardInterrupt: 

In [39]:
# Example usage
ttl_file_path = '../csvw/walmart_products.ttl'
spotlight_api_url = 'https://api.dbpedia-spotlight.org/en/annotate'
output_ttl_file = '../csvw/walmart_products_annotated.ttl'

# Apply to the first 5 triplets only
annotate(ttl_file_path, spotlight_api_url, output_ttl_file)

UnicodeEncodeError: 'charmap' codec can't encode character '\xe9' in position 69: character maps to <undefined>

In [2]:
%pip install pyspotlight

Note: you may need to restart the kernel to use updated packages.  Downloading pyspotlight-0.7.2-py2.py3-none-any.whl (15 kB)




[notice] A new release of pip is available: 23.1.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import spotlight
annotations = spotlight.annotate('https://api.dbpedia-spotlight.org/en/annotate?types={DBpedia:Food}', "Hummus, Dips, & Salsa", confidence=0.3)


print("cat ", annotations[0]['URI'])

cat  http://dbpedia.org/resource/Hummus
